# Perceptrón para Análisis de Sentimiento en Español

## Objetivo

En esta actividad vas a implementar desde cero un modelo de Perceptrón en Python usando solo NumPy. Vamos a entrenarlo con frases breves escritas en español rioplatense para que aprenda a reconocer si una frase tiene un sentimiento positivo o negativo.

El objetivo es entender cómo funciona una neurona artificial básica, cómo se ajustan sus pesos durante el aprendizaje, y cómo puede hacer predicciones en base a un conjunto pequeño de frases.

### ¿Qué es un perceptrón?

El perceptrón es el modelo más simple de neurona artificial. Fue propuesto en 1958 por Frank Rosenblatt y representa la base fundamental de las redes neuronales modernas.

**Funcionamiento básico:**
1. Recibe múltiples entradas (features)
2. Multiplica cada entrada por un peso
3. Suma todos los resultados y agrega un sesgo (bias)
4. Aplica una función de activación para decidir la salida

Es un modelo lineal que solo puede aprender patrones que sean linealmente separables.

## 1. Datos de entrenamiento

Vamos a usar un pequeño conjunto de frases etiquetadas como positivas (1) o negativas (0). Las frases son simples como las que podríamos encontrar en una conversación cotidiana en Argentina.

También definimos un vocabulario básico de palabras clave que aparecen con frecuencia y que nos pueden ayudar a inferir el sentimiento.

In [1]:
import numpy as np

# Frases con su etiqueta de sentimiento (1 = positivo, 0 = negativo)
frases = [
    "Amo el verano en Buenos Aires",
    "No me gusta el tráfico matutino",
    "Este asado está espectacular",
    "Qué bajón, perdí el colectivo",
    "Me encanta salir los domingos",
    "Detesto el calor húmedo"
]

etiquetas = np.array([1, 0, 1, 0, 1, 0])  # Etiquetas correspondientes

# Vocabulario manual con palabras claves de carga emocional
vocabulario = ["amo", "no", "gusta", "asado", "espectacular", "bajón", "perdí", "detesto", "calor", "húmedo"]

print(f"Total de frases: {len(frases)}")
print(f"Vocabulario: {len(vocabulario)} palabras clave")
print(f"\nPrimera frase: '{frases[0]}' → Sentimiento: {'Positivo' if etiquetas[0] == 1 else 'Negativo'}")

Total de frases: 6
Vocabulario: 10 palabras clave

Primera frase: 'Amo el verano en Buenos Aires' → Sentimiento: Positivo


## 2. Representación numérica: Vectorización de frases

Las redes neuronales no pueden procesar texto directamente. Necesitamos convertir cada frase en un vector numérico.

### Bag of Words (Bolsa de Palabras)

Vamos a usar una representación simple llamada **bag of words**: para cada frase, creamos un vector binario que indica si cada palabra del vocabulario aparece (1) o no (0) en la frase.

**Ejemplo:**
- Vocabulario: ["amo", "no", "gusta", "asado"]
- Frase: "Amo el asado"
- Vector: [1, 0, 0, 1] (tiene "amo" y "asado", no tiene "no" ni "gusta")

**Limitación importante:** Este método no captura el orden de las palabras ni el contexto. "No me gusta" y "me gusta" tendrían vectores muy similares.

In [2]:
def vectorizar(frase, vocabulario):
    """
    Convierte una frase en un vector binario según el vocabulario.

    Args:
        frase: String con la frase a vectorizar
        vocabulario: Lista de palabras clave

    Returns:
        Array de numpy con 1s y 0s
    """
    tokens = frase.lower().split()
    return np.array([1 if palabra in tokens else 0 for palabra in vocabulario])

# Aplicamos la función a todas las frases
X = np.array([vectorizar(frase, vocabulario) for frase in frases])

print("Matriz de features (X):")
print(f"Forma: {X.shape} (6 frases × 10 features)\n")
print("Vectores generados:")
for i, (frase, vector) in enumerate(zip(frases, X)):
    print(f"Frase {i+1}: {vector} → Sentimiento: {'Positivo' if etiquetas[i] == 1 else 'Negativo'}")

Matriz de features (X):
Forma: (6, 10) (6 frases × 10 features)

Vectores generados:
Frase 1: [1 0 0 0 0 0 0 0 0 0] → Sentimiento: Positivo
Frase 2: [0 1 1 0 0 0 0 0 0 0] → Sentimiento: Negativo
Frase 3: [0 0 0 1 1 0 0 0 0 0] → Sentimiento: Positivo
Frase 4: [0 0 0 0 0 0 1 0 0 0] → Sentimiento: Negativo
Frase 5: [0 0 0 0 0 0 0 0 0 0] → Sentimiento: Positivo
Frase 6: [0 0 0 0 0 0 0 1 1 1] → Sentimiento: Negativo


## 3. Definición del modelo: el Perceptrón

Un perceptrón es una función matemática que:

1. **Multiplica** cada entrada por un peso (weight)
2. **Suma** los resultados y agrega un sesgo (bias)
3. **Aplica** una función de activación para decidir si "dispara" o no

### Fórmula matemática:

```
z = (x₁ × w₁) + (x₂ × w₂) + ... + (xₙ × wₙ) + bias
salida = función_activación(z)
```

### Función de activación escalón:

```
Si z > 0 → salida = 1 (positivo)
Si z ≤ 0 → salida = 0 (negativo)
```

Vamos a inicializar los pesos aleatoriamente y entrenar el modelo para que aprenda de los errores.

In [3]:
# Inicializamos pesos y bias con valores aleatorios pequeños
np.random.seed(42)  # Para reproducibilidad
pesos = np.random.randn(len(vocabulario))
bias = 0.0

print("Pesos iniciales (aleatorios):")
for i, (palabra, peso) in enumerate(zip(vocabulario, pesos)):
    print(f"  w[{palabra}] = {peso:.3f}")
print(f"\nBias inicial: {bias}")

def activar(suma):
    """
    Función de activación escalón (step function).
    Devuelve 1 si la suma es positiva, 0 en caso contrario.
    """
    return 1 if suma > 0 else 0

def predecir(x):
    """
    Hace una predicción para un vector de entrada x.

    Args:
        x: Vector de features (array de numpy)

    Returns:
        Predicción: 0 o 1
    """
    suma = np.dot(x, pesos) + bias
    return activar(suma)

print("\nFunciones definidas: activar() y predecir()")

Pesos iniciales (aleatorios):
  w[amo] = 0.497
  w[no] = -0.138
  w[gusta] = 0.648
  w[asado] = 1.523
  w[espectacular] = -0.234
  w[bajón] = -0.234
  w[perdí] = 1.579
  w[detesto] = 0.767
  w[calor] = -0.469
  w[húmedo] = 0.543

Bias inicial: 0.0

Funciones definidas: activar() y predecir()


## 4. Entrenamiento del modelo

Ahora vamos a entrenar el perceptrón ajustando los pesos según los errores que comete.

### Regla de aprendizaje del perceptrón:

Para cada ejemplo:
1. Calcular la predicción
2. Calcular el error: `error = etiqueta_real - predicción`
3. Si hay error (≠ 0), ajustar los pesos:
   - `peso_nuevo = peso_viejo + (tasa_aprendizaje × error × entrada)`
   - `bias_nuevo = bias_viejo + (tasa_aprendizaje × error)`

### Parámetros de entrenamiento:

- **Tasa de aprendizaje**: Controla qué tan grande es cada ajuste (0.1 es un valor común)
- **Épocas**: Cantidad de veces que recorremos todo el dataset

El entrenamiento se detiene cuando el modelo clasifica correctamente todos los ejemplos o llega al máximo de épocas.

In [5]:
# Parámetros de entrenamiento
tasa_aprendizaje = 0.1
epocas = 50

print("="*60)
print("INICIANDO ENTRENAMIENTO")
print("="*60)
print(f"Tasa de aprendizaje: {tasa_aprendizaje}")
print(f"Épocas máximas: {epocas}")
print(f"Ejemplos de entrenamiento: {len(X)}\n")

# Bucle de entrenamiento
for epoca in range(epocas):
    errores = 0

    # Recorremos cada ejemplo
    for i in range(len(X)):
        x_i = X[i]
        y_real = etiquetas[i]
        y_pred = predecir(x_i)
        error = y_real - y_pred

        # Si hay error, ajustamos pesos y bias
        if error != 0:
            pesos += tasa_aprendizaje * error * x_i
            bias += tasa_aprendizaje * error
            errores += 1

    print(f"Época {epoca + 1:2d}: Errores = {errores}")

    # Si no hay errores, el modelo convergió
    if errores == 0:
        print(f"\nConvergencia alcanzada en época {epoca + 1}")
        break

print("\n" + "="*60)
print("ENTRENAMIENTO FINALIZADO")
print("="*60)

INICIANDO ENTRENAMIENTO
Tasa de aprendizaje: 0.1
Épocas máximas: 50
Ejemplos de entrenamiento: 6

Época  1: Errores = 1
Época  2: Errores = 2
Época  3: Errores = 2
Época  4: Errores = 1
Época  5: Errores = 2
Época  6: Errores = 2
Época  7: Errores = 0

Convergencia alcanzada en época 7

ENTRENAMIENTO FINALIZADO


## 5. Análisis de los pesos aprendidos

Después del entrenamiento, los pesos nos indican qué tan importante es cada palabra para determinar el sentimiento.

- **Pesos positivos**: Palabras asociadas a sentimiento positivo
- **Pesos negativos**: Palabras asociadas a sentimiento negativo
- **Pesos cercanos a 0**: Palabras poco informativas

In [6]:
print("Pesos aprendidos después del entrenamiento:\n")
print(f"{'Palabra':<15} {'Peso':>10} {'Interpretación'}")
print("-" * 50)

for palabra, peso in sorted(zip(vocabulario, pesos), key=lambda x: x[1], reverse=True):
    if peso > 0.1:
        interpretacion = "→ Positivo"
    elif peso < -0.1:
        interpretacion = "→ Negativo"
    else:
        interpretacion = "→ Neutral"

    print(f"{palabra:<15} {peso:>10.3f}  {interpretacion}")

print(f"\nBias final: {bias:.3f}")
print("\nEl bias representa el umbral base del modelo.")

Pesos aprendidos después del entrenamiento:

Palabra               Peso Interpretación
--------------------------------------------------
asado                1.523  → Positivo
amo                  0.497  → Positivo
detesto              0.367  → Positivo
gusta                0.248  → Positivo
húmedo               0.143  → Positivo
perdí               -0.121  → Negativo
bajón               -0.234  → Negativo
espectacular        -0.234  → Negativo
no                  -0.538  → Negativo
calor               -0.869  → Negativo

Bias final: 0.100

El bias representa el umbral base del modelo.


## 6. Prueba con nuevas frases

Ahora vamos a ver cómo se comporta nuestro perceptrón con frases nuevas que no vio durante el entrenamiento.

Esta es la verdadera prueba: ¿puede generalizar lo que aprendió a casos nuevos?

In [7]:
# Frases nuevas para testeo
frases_prueba = [
    "No aguanto este calor",
    "Qué hermoso día para pasear",
    "Detesto levantarme temprano"
]

print("="*60)
print("PREDICCIONES EN FRASES NUEVAS")
print("="*60)

# Vectorizamos las frases nuevas
X_prueba = np.array([vectorizar(frase, vocabulario) for frase in frases_prueba])
predicciones = [predecir(x) for x in X_prueba]

# Mostramos los resultados
for i, (frase, pred, vector) in enumerate(zip(frases_prueba, predicciones, X_prueba), 1):
    resultado = "Positivo" if pred == 1 else "Negativo"
    print(f"\nFrase {i}: '{frase}'")
    print(f"  Vector: {vector}")
    print(f"  Predicción: {resultado}")

    # Mostramos qué palabras del vocabulario detectó
    palabras_detectadas = [vocabulario[j] for j in range(len(vocabulario)) if vector[j] == 1]
    if palabras_detectadas:
        print(f"  Palabras clave detectadas: {', '.join(palabras_detectadas)}")
    else:
        print(f"  No se detectaron palabras del vocabulario")

print("\n" + "="*60)

PREDICCIONES EN FRASES NUEVAS

Frase 1: 'No aguanto este calor'
  Vector: [0 1 0 0 0 0 0 0 1 0]
  Predicción: Negativo
  Palabras clave detectadas: no, calor

Frase 2: 'Qué hermoso día para pasear'
  Vector: [0 0 0 0 0 0 0 0 0 0]
  Predicción: Positivo
  No se detectaron palabras del vocabulario

Frase 3: 'Detesto levantarme temprano'
  Vector: [0 0 0 0 0 0 0 1 0 0]
  Predicción: Positivo
  Palabras clave detectadas: detesto



## 7. Reflexión final

### ¿Qué aprendimos?

1. **Funcionamiento de una neurona artificial básica**: El perceptrón es el bloque fundamental de las redes neuronales. Aprendimos cómo combina entradas ponderadas y aplica una función de activación.

2. **Proceso de entrenamiento**: Vimos cómo un modelo aprende ajustando sus pesos iterativamente basándose en los errores que comete. Este principio se extiende a redes neuronales más complejas.

3. **Representación de texto**: Usamos bag-of-words, una técnica simple pero efectiva para convertir texto en números que las máquinas pueden procesar.

### Limitaciones observadas

1. **No considera el orden**: "No me gusta" vs "Me gusta, no" se representan igual
2. **Vocabulario limitado**: Solo conoce las palabras que definimos manualmente
3. **Modelo lineal**: Solo puede aprender patrones linealmente separables
4. **Sin contexto**: No entiende sarcasmo, ironía o matices del lenguaje
5. **Dataset pequeño**: Con solo 6 ejemplos, la generalización es limitada

### ¿Qué sigue?

En el próximo laboratorio, vamos a ver cómo las **redes neuronales multicapa** (MLP) pueden capturar patrones más complejos usando múltiples perceptrones organizados en capas. Esto nos va a permitir:

- Aprender representaciones no lineales
- Capturar interacciones entre features
- Mejorar la capacidad de generalización

Más adelante veremos cómo las **redes recurrentes** (LSTM) pueden procesar el orden de las palabras y, finalmente, cómo los **Transformers** revolucionaron el procesamiento de lenguaje natural.